In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import psycopg2
from bs4 import BeautifulSoup as bp
import time
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from shapely.geometry import Point

In [4]:
import getpass
import os

postgres_url = getpass.getpass("Enter your postgresql url: ")

In [5]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
REQUEST_HEADER = {
    'User-Agent': USER_AGENT,
    'Accept-language': 'vi-vn',
}

In [6]:
options=Options()
options.add_argument(f"user-agent={USER_AGENT}")
driver=webdriver.Chrome(options=options)

In [7]:
driver.get('https://www.google.com/maps')

In [23]:
def get_location(address):
    # Click the search box input element
    find = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
    find.click()
    find.clear()
    time.sleep(1)

# Input the address
    find.send_keys(address)
    time.sleep(1)

#right_click to get the box with location
    maps = driver.find_element(By.XPATH, '//*[@id="scene"]/div[3]/canvas[1]')
    time.sleep(2)
    right_click = ActionChains(driver).context_click(maps)
    right_click.perform()
    time.sleep(1)  

# Retrieve the text from the location box
    location_box = driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[1]/div/div')
    text = location_box.text
    lat_text,long_text = text.split(",")
    latitude=float(lat_text)
    longitude=float(long_text)
    print(f"location: latitude {latitude}, longitude: {longitude}")
    return {'latitude': latitude, 'longitude': longitude}

In [31]:
def insert_location(id,location):
    print(f"insert location {id} into db")
    if isinstance(location, dict) and 'longitude' in location and 'latitude' in location:
        point = Point(location['longitude'], location['latitude'])
        point_wkt = point.wkt
        conn = psycopg2.connect(postgres_url)
        cur = conn.cursor()
        cur.execute("""SET search_path TO travel_database, public;""")
        query="""
        UPDATE Hotel SET location = ST_GeomFromText(%s, 4326)
        WHERE hotel_id = %s;"""
        cur.execute(query, (point_wkt,id))
        conn.commit()
        cur.close()
        conn.close()
        print(f"updated location for {id}")
    else:
        print("error: location is not dict")

In [29]:
def get_address():
    print(f"taking address of NULL places!!!")
    conn=psycopg2.connect(postgres_url)
    cur=conn.cursor()
    cur.execute("""SET search_path TO travel_database, public;""")
    query="""
    SELECT address,hotel_id from hotel WHERE location is NULL"""
    cur.execute(query,)
    conn.commit()
    a= cur.fetchall()
    cur.close()
    conn.close()
    print(f"found {len(a)} places")
    return a

In [33]:
if __name__=='__main__':
    a=get_address()
    for row in a:
        address=row[0]
        id=row[1]
        location=get_location(address)
        insert_location(id,location)
        time.sleep(2)


taking address of NULL places!!!
found 1090 places
location: latitude21.01051, longitude:105.80248
insert location 1248 into db
updated location for 1248
location: latitude21.01051, longitude:105.80248
insert location 2 into db
updated location for 2
location: latitude21.01051, longitude:105.80248
insert location 652 into db
updated location for 652
location: latitude21.01051, longitude:105.80248
insert location 1252 into db
updated location for 1252
location: latitude21.01051, longitude:105.80248
insert location 1258 into db
updated location for 1258
location: latitude21.01051, longitude:105.80248
insert location 35 into db
updated location for 35
location: latitude21.01051, longitude:105.80248
insert location 37 into db
updated location for 37
location: latitude21.01051, longitude:105.80248
insert location 2526 into db
updated location for 2526
location: latitude21.01051, longitude:105.80248
insert location 43 into db
updated location for 43
location: latitude21.01051, longitude:105.

KeyboardInterrupt: 